# Genji

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = 'https://ja.wikisource.org/wiki/%E6%BA%90%E6%B0%8F%E7%89%A9%E8%AA%9E_(%E6%B8%8B%E8%B0%B7%E6%A0%84%E4%B8%80%E6%A0%A1%E8%A8%82)'

In [2]:
base_soup = BeautifulSoup(requests.get(base_url).text)
ul = base_soup.find_all('ul')[1]

chapter_dict = {}
for li in ul.find_all('li'):
    chapter_dict[li.text] = f'https://ja.wikisource.org/{li.a["href"]}'


tog = pd.DataFrame.from_dict(chapter_dict, orient='index').reset_index().rename(columns={'index':'chapter',0:'link'})

In [3]:
def get_text(link):
    text = ''
    soup = BeautifulSoup(requests.get(link).text)
    for p in soup.find_all('p')[3:]:
        text += p.text.strip()
    return text

In [4]:
from tqdm import tqdm
tqdm.pandas()

tog['text'] = tog.link.progress_apply(get_text)

100%|██████████| 54/54 [00:15<00:00,  3.49it/s]


In [18]:
from konoha import SentenceTokenizer
tokenizer = SentenceTokenizer()

tog['sents'] = tog['text'].apply(tokenizer.tokenize)
tog_sents = tog[['chapter','sents']]
tog_sents = tog_sents.explode('sents').reset_index(drop=True)
tog_sents

,chapter,sents
0,桐壺,いづれの御時にか、女御、更衣あまたさぶらひたまひけるなかに、いとやむごとなき際にはあらぬが、...
1,桐壺,はじめより我はと思ひ上がりたまへる御方がた、めざましきものにおとしめ嫉みたまふ。
2,桐壺,同じほど、それより下臈の更衣たちは、ましてやすからず。
3,桐壺,朝夕の宮仕へにつけても、人の心をのみ動かし、恨みを負ふ積もりにやありけむ、いと篤しくなりゆき...
4,桐壺,上達部、上人なども、あいなく目を側めつつ、「いとまばゆき人の御おぼえなり。唐土にも、かかる事...
...,...,...
11055,夢浮橋,「いかが聞こえむ」など責められて、「心地のかき乱るやうにしはべるほど、ためらひて、今聞こえむ...
11056,夢浮橋,主人ぞ、この君に物語すこし聞こえて、「もののけにやおはすらむ。例のさまに見えたまふ折なく、悩...
11057,夢浮橋,所につけてをかしき饗応などしたれど、幼き心地は、そこはかとなくあわてたる心地して、「わざと奉...
11058,夢浮橋,人知れずゆかしき御ありさまをも、え見ずなりぬるを、おぼつかなく口惜しくて、心ゆかずながら参りぬ。


In [8]:
import spacy
nlp = spacy.load('ja_core_news_sm')

In [9]:
from spacy.tokens import DocBin

def spacyify(nlp, df,col_name,f_name='spacy_model_output'):
    texts = df[col_name].to_list()
    doc_bin = DocBin()
    for doc in tqdm(nlp.pipe(texts), total=len(texts)):
        doc_bin.add(doc)
    bytes_data = doc_bin.to_bytes()

    f = open(f'serialized_data/{f_name}','wb')
    f.write(bytes_data)
    f.close()

In [19]:
spacyify(nlp, tog_sents, 'sents')

100%|██████████| 11060/11060 [01:01<00:00, 181.11it/s]


In [20]:
tog_sents.to_csv('tog_sents.csv')

# Manyoshu

In [43]:
soup = BeautifulSoup(requests.get('https://jti.lib.virginia.edu/japanese/manyoshu/Man1Yos.html').text)
aoi = soup.center.td

In [59]:
from bs4 import NavigableString, Tag
for t in aoi:
    #if isinstance(tag.previous_element, NavigableString) and (tag.name == 'blockquote'):
    print('-----')
    print('prev:', t.previous_element, type(t.previous_element))
    print('curr:',t, type(t))
    print('next:',t.next_element, type(t.next_element))
    print('-----')

-----
prev: <td>
<center><h2><font color="#800000"><a name="第一巻">第一巻</a></font></h2></center>
<center><h1><font color="#800000"><b>雜歌</b></font></h1></center>
<br/><hr color="#800000" noshade="" size="2" width="80%"/><br/><br/>
<center><h2><font color="#800000"><a name="1">1</a></font></h2></center>
<p><font size="-1"><blockquote>[題詞]泊瀬朝倉宮御宇天皇代  [&lt;大&gt;泊瀬稚武天皇] / 天皇御製歌</blockquote></font></p><br/>
<p>[原文]篭毛與  美篭母乳  布久思毛與  美夫君志持  此岳尓  菜採須兒  家吉閑名  告&lt;紗&gt;根  虚見津  山跡乃國者  押奈戸手  吾許曽居  師&lt;吉&gt;名倍手  吾己曽座  我&lt;許&gt;背齒 告目  家呼毛名雄母</p>
<p>[訓読]篭もよ  み篭持ち  堀串もよ  み堀串持ち  この岡に  菜摘ます子  家聞かな  告らさね  そらみつ  大和の国は  おしなべて  我れこそ居れ  しきなべて  我れこそ座せ  我れこそば  告らめ  家をも名をも</p>
<p>[仮名],こもよ,みこもち,ふくしもよ,みぶくしもち,このをかに,なつますこ,いへきかな,のらさね,そらみつ,やまとのくには,おしなべて,われこそをれ,しきなべて,われこそませ,われこそば,のらめ,いへをもなをも</p>
<blockquote><font size="-1">
<p>[左注]</p></font>
<font size="-1">
<p>[校異]雑歌  [元][紀]  &lt;&gt; / 太 <img src="/japanese/manyoshu/rarr.gif"/> 大  [紀][冷][文] / 吉  [玉小琴](塙)(楓)  告 /沙 <img src="/japanese/manyoshu/rarr.gif"/> 紗 [元][類][冷]

In [152]:
import collections
poem_dict = collections.defaultdict(list)
for i,t in enumerate(aoi):
    if (isinstance(t.previous_element, NavigableString)) and (isinstance(t.next_element, Tag)) and (t.name == 'p'):
        for ele in t.next_siblings:
            if not isinstance(ele, NavigableString):
                if (ele.name == 'p') and (isinstance(ele.next_sibling, NavigableString)) and (ele.next_sibling.next_sibling.name == 'blockquote'):
                    poem_dict[f'{t.blockquote.text.strip()}_{i}'].append(ele.text)
                    break
                else:
                    poem_dict[f'{t.blockquote.text.strip()}_{i}'].append(ele.text)
        poem_dict[f'{t.blockquote.text.strip()}_{i}'] = [line for line in poem_dict[f'{t.blockquote.text.strip()}_{i}'] if len(line)>0]
poem_dict = dict(poem_dict)

In [158]:
pd.DataFrame.from_dict(poem_dict, orient='index').reset_index().rename(columns={'index':'title',0:'first',1:'second',2:'third'})

,title,first,second,third
0,[題詞]泊瀬朝倉宮御宇天皇代 [<大>泊瀬稚武天皇] / 天皇御製歌_12,[原文]篭毛與 美篭母乳 布久思毛與 美夫君志持 此岳尓 菜採須兒 家吉閑名 ...,[訓読]篭もよ み篭持ち 堀串もよ み堀串持ち この岡に 菜摘ます子 家聞かな ...,"[仮名],こもよ,みこもち,ふくしもよ,みぶくしもち,このをかに,なつますこ,いへきかな,の..."
1,[題詞]高市岡本宮御宇天皇代 [息長足日廣額天皇] / 天皇登香具山望國之時御製歌_28,[原文]山常庭 村山有等 取與呂布 天乃香具山 騰立 國見乎為者 國原波 煙立...,[訓読]大和には 群山あれど とりよろふ 天の香具山 登り立ち 国見をすれば 国...,"[仮名],やまとには,むらやまあれど,とりよろふ,あめのかぐやま,のぼりたち,くにみをすれば..."
2,[題詞]天皇遊猟内野之時中皇命使間人連老獻歌_44,[原文]八隅知之 我大王乃 朝庭 取撫賜 夕庭 伊縁立之 御執乃 梓弓之 奈...,[訓読]やすみしし 我が大君の 朝には 取り撫でたまひ 夕には い寄り立たしし ...,"[仮名],やすみしし,わがおほきみの,あしたには,とりなでたまひ,ゆふへには,いよりたたしし..."
3,[題詞]（天皇遊猟内野之時中皇命使間人連老獻歌）反歌_60,[原文]玉尅春 内乃大野尓 馬數而 朝布麻須等六 其草深野,[訓読]たまきはる宇智の大野に馬並めて朝踏ますらむその草深野,"[仮名],たまきはる,うちのおほのに,うまなめて,あさふますらむ,そのくさふかの"
4,[題詞]幸讃岐國安益郡之時軍王見山作歌_76,[原文]霞立 長春日乃 晩家流 和豆肝之良受 村肝乃 心乎痛見 奴要子鳥 卜歎...,[訓読]霞立つ 長き春日の 暮れにける わづきも知らず むらきもの 心を痛み ぬ...,"[仮名],かすみたつ,ながきはるひの,くれにける,わづきもしらず,むらきもの,こころをいたみ..."
...,...,...,...,...
79,[題詞]（或本従藤原<京>遷于寧樂宮時歌）反歌_1276,[原文]青丹吉 寧樂乃家尓者 万代尓 吾母将通 忘跡念勿,[訓読]あをによし奈良の家には万代に我れも通はむ忘ると思ふな,"[仮名],あをによし,ならのいへには,よろづよに,われもかよはむ,わするとおもふな"
80,[題詞]和銅五年壬子夏四月遣長田王于伊勢齊宮時山邊御井<作>歌_1292,[原文]山邊乃 御井乎見我弖利 神風乃 伊勢處女等 相見鶴鴨,[訓読]山辺の御井を見がてり神風の伊勢娘子どもあひ見つるかも,"[仮名],やまのへの,みゐをみがてり,かむかぜの,いせをとめども,あひみつるかも"
81,[題詞]（和銅五年壬子夏四月遣長田王于伊勢齊宮時山邊御井<作>歌）_1308,[原文]浦佐夫流 情佐麻<祢>之 久堅乃 天之四具礼能 流相見者,[訓読]うらさぶる心さまねしひさかたの天のしぐれの流らふ見れば,"[仮名],うらさぶる,こころさまねし,ひさかたの,あめのしぐれの,ながらふみれば"
82,[題詞]（和銅五年壬子夏四月遣長田王于伊勢齊宮時山邊御井<作>歌）_1324,[原文]海底 奥津白波 立田山 何時鹿越奈武 妹之當見武,[訓読]海の底沖つ白波龍田山いつか越えなむ妹があたり見む,"[仮名],わたのそこ,おきつしらなみ,たつたやま,いつかこえなむ,いもがあたりみむ"


In [203]:
def get_texts(url):
    soup = BeautifulSoup(requests.get(url).text)
    aoi = soup.center.td

    poem_dict = collections.defaultdict(list)
    for i,t in enumerate(aoi):
        if (isinstance(t.previous_element, NavigableString)) and (isinstance(t.next_element, Tag)) and (t.name == 'p'):
            for ele in t.next_siblings:
                if not isinstance(ele, NavigableString):
                    if (ele.name == 'p') and (isinstance(ele.next_sibling, NavigableString)) and (ele.next_sibling.next_sibling.name == 'blockquote'):
                        poem_dict[f'{t.blockquote.text.strip()}_{i}'].append(ele.text[4:])
                        break
                    else:
                        poem_dict[f'{t.blockquote.text.strip()}_{i}'].append(ele.text[4:])
            poem_dict[f'{t.blockquote.text.strip()}_{i}'] = [line for line in poem_dict[f'{t.blockquote.text.strip()}_{i}'] if len(line)>0]
    poem_dict = dict(poem_dict)

    return pd.DataFrame.from_dict(poem_dict, orient='index').reset_index().rename(columns={'index':'title',0:'manyogana',1:'kundoku',2:'kana'})

In [206]:
DFs = []
for i in range(1, 21):
    if i < 10:
        DFs.append(get_texts(f'https://jti.lib.virginia.edu/japanese/manyoshu/Man{i}Yos.html'))
    else:
        DFs.append(get_texts(f'https://jti.lib.virginia.edu/japanese/manyoshu/Man{i}Yo.html'))
manyoshu = pd.concat(DFs).dropna()

In [213]:
ex = manyoshu.loc[manyoshu.manyogana.str.contains('空蝉'), 'manyogana'].iloc[1]

In [214]:
spacy_doc = nlp(ex)
search = '空蝉'
for token in spacy_doc:
    if ((token.lemma_.lower() == search) or (token.lemma_.title() == search)) and (token.head.pos_ != 'PUNCT'):
        print(
            (token.text, token.pos_), 
            (token.head.text, token.head.pos_)
            )
        print(spacy_doc.text)

('空蝉', 'NOUN') ('勝者', 'NOUN')
空蝉師  神尓不勝者  離居而  朝嘆君  放居而  吾戀君  玉有者  手尓巻持而  衣有者  脱時毛無  吾戀  君曽伎賊乃夜  夢所見鶴


In [215]:
spacyify(nlp, manyoshu, 'manyogana', f_name='manyoshu_spacy_output')

100%|██████████| 4560/4560 [00:13<00:00, 331.13it/s]


In [216]:
manyoshu.to_csv('manyoshu.csv')

# Kokinshu

In [2]:
from bs4 import BeautifulSoup
from bs4 import NavigableString, Tag
import requests
import re
tei = BeautifulSoup(requests.get('https://jti.lib.virginia.edu/japanese/kokinshu/kikokin.html').text).teiheader

In [41]:
import collections

poem_dict = collections.defaultdict(str)
for center in tei.find_all('center')[1:]:
    if center.text.strip().isdigit():
        num = center.text.strip()
        print(num)
        sib_list = list(center.next_siblings)
        if sib_list[3].name == 'center': # if title
            if sib_list[7].name == 'blockquote': # if blockquote
                print(sib_list[3].text.strip())
                print(sib_list[7].text.strip())
                print(str(sib_list[10]).strip())
            else:
                print(sib_list[3].text.strip())
                print(str(sib_list[6]).strip())
        else:
            if sib_list[3].name == 'blockquote':
                print(sib_list[3].text.strip())
                print(str(sib_list[7]).strip())
            else:
                print(str(sib_list[3]).strip())
        print('---')
        # if sib_list[3].name == 'blockquote':
        #     poem_dict[f'{center.text.strip()}_{num}'] += f'{sib_list[3].text.strip()}\n'
        #     poem_dict[f'{center.text.strip()}_{num}'] += str(sib_list[6].strip())
        # else:
        #     poem_dict[f'{center.text.strip()}_{num}'] += str(sib_list[3]).strip()

1
在原元方
ふるとしに春たちける日よめる
としのうちに春はきにけりひととせをこぞとやいはむことしとやいはむ
---
2
紀貫之
はるたちける日よめる
袖ひちてむすびし水のこほれるを春立つけふの風やとくらむ
---
3
よみ人しらず
題しらず
春霞たてるやいづこみよしののよしのの山に雪はふりつつ
---
4
二条のきさきのはるのはじめの御うた
<br/>
---
5
よみ人しらず
題しらず
梅がえにきゐるうぐひすはるかけてなけどもいまだ雪はふりつつ
---
6
素性法師
雪の木にふりかかれるをよめる
春立てば花とや見らむ白雪のかかれる枝にうぐひすぞなく
---
7
よみ人しらず
題しらず
心ざしふかくそめてし折りければきえあへぬ雪の花と見ゆらむ
---
8
文屋やすひで
二条のきさきのとう宮のみやすんどころときこえけ   
る時、正月三日おまへにめして、おほせごとあるあひだに、日はてりながら雪のかしら   
にふりかかりけるをよませ給ひける

---
9
きのつらゆき
ゆきのふりけるをよめる
霞たちこのめもはるの雪ふれば花なきさとも花ぞちりける
---
10
ふぢはらのことなほ
春のはじめによめる
はるやとき花やおそきとききわかむ鶯だにもなかずもあるかな
---
11
みぶのただみね
はるのはじめのうた
春きぬと人はいへどもうぐひすのなかぬかぎりはあらじとぞ思ふ
---
12
源まさずみ
寛平御時きさいの宮のうたあはせのうた
谷風にとくるこほりのひまごとにうちいづる浪や春のはつ花
---
13
紀とものり
花のかを風のたよりにたぐへてぞ鶯さそふしるべにはやる
---
14
大江千里
うぐひすの谷よりいづるこゑなくは春くることをたれかしらまし
---
15
在原棟梁
春たてど花もにほはぬ山ざとはものうかるねに鶯ぞなく
---
16
よみ人しらず
題しらず
野辺ちかくいへゐしせればうぐひすのなくなるこゑはあさなあさなきく
---
17

<br/>
---
18

<br/>
---
19

<br/>
---
20

<br/>
---
21
仁和のみかどみこにおましましける時に、人にわか   
なたまひける御うた
<br/>
---
22
つらゆき
哥たてまつれとおほせられし時よみてたてまつれる

---
23
在原行平朝臣
題しらず
はるのきるかすみの衣ぬきをうす

IndexError: list index out of range

In [23]:
dict(poem_dict)

{'在原元方_1': 'ふるとしに春たちける日よめる\nとしのうちに春はきにけりひととせをこぞとやいはむことしとやいはむ',
 '紀貫之_2': 'はるたちける日よめる\n袖ひちてむすびし水のこほれるを春立つけふの風やとくらむ',
 'よみ人しらず_3': '題しらず\n春霞たてるやいづこみよしののよしのの山に雪はふりつつ',
 'よみ人しらず_5': '題しらず\n梅がえにきゐるうぐひすはるかけてなけどもいまだ雪はふりつつ',
 '素性法師_6': '雪の木にふりかかれるをよめる\n春立てば花とや見らむ白雪のかかれる枝にうぐひすぞなく',
 'よみ人しらず_7': '題しらず\n心ざしふかくそめてし折りければきえあへぬ雪の花と見ゆらむ',
 '文屋やすひで_8': '二条のきさきのとう宮のみやすんどころときこえけ   \nる時、正月三日おまへにめして、おほせごとあるあひだに、日はてりながら雪のかしら   \nにふりかかりけるをよませ給ひける\n',
 'きのつらゆき_9': 'ゆきのふりけるをよめる\n霞たちこのめもはるの雪ふれば花なきさとも花ぞちりける',
 'ふぢはらのことなほ_10': '春のはじめによめる\nはるやとき花やおそきとききわかむ鶯だにもなかずもあるかな',
 'みぶのただみね_11': 'はるのはじめのうた\n春きぬと人はいへどもうぐひすのなかぬかぎりはあらじとぞ思ふ',
 '源まさずみ_12': '寛平御時きさいの宮のうたあはせのうた\n谷風にとくるこほりのひまごとにうちいづる浪や春のはつ花',
 '紀とものり_13': '<br/>',
 '大江千里_14': '<br/>',
 '在原棟梁_15': '<br/>',
 'よみ人しらず_16': '題しらず\n野辺ちかくいへゐしせればうぐひすのなくなるこゑはあさなあさなきく',
 'つらゆき_22': '哥たてまつれとおほせられし時よみてたてまつれる\n',
 '在原行平朝臣_23': '題しらず\nはるのきるかすみの衣ぬきをうすみ山風にこそみだるべらなれ',
 '源むねゆきの朝臣_24': '寛平御時きさいの宮の哥合によめる\nときはなる松のみどりも春くれば今ひとしほの色まさりけり',
 'つらゆき_25': '哥たてまつれとおほせられし時によみてたてまつれ   \nる\n',
 '僧正